# Visual Wake World

Visual Wake Words represents a common microcontroller vision use case, where the goal is to identify whether a person or an object of interest is actually present in an image or not. For example, consider a smart doorbell: it recognizes when someone shows up at a door and if it does not recognize the person it might send us an alert.

<img src="./images/03.png" width="500">

We have an image sensor, which generates RGB channels of input data that we will be feeding into a neural network.
Until now, we have trained our neural networks from scratch, however we can shortcut that process in order to accelerate the training time by reusing some of the information that a network has retained from prior experiences.


If we look at a simple little image, it is around 224 by 224 (the typically resolution of TinyML applications are dealing with). The camera can have a higher resolution, however we normally down scale it into small images because neural networks are very good at picking up features, even at those small dimensions. That means we have 224 by 224 pixels times 3 (because there are three channels, since it's a color image). If we assume that each of those pixels is occupying a floating point value (4 bytes) then it comes out to be around 602,000 bytes. 

Now let's talk about what this means in the context of uploading this data to the cloud. Some  typical characteristics of network connection are a ping of about 25 ms (latency just to be able to send something from our local gateway and to get a response back), download speed (often much faster than upload speed) about 35 Mbps and upload speed bout 4.62 Mbs.

<img src="./images/01.png" width="500">

So, our task is going to take one second to upload, 10 ms to do the inference (fast on the datacenter)and another 100 milliseconds to come back to the machine to provide the answer. That's about 1.2 seconds.

<img src="./images/02.png" width="500">

This can be pretty bad, because in certain cases, if the object of interest moves away within the one second, by the time you haeve gotten a decision the image might have changed. And, in general, this latency is not tolerable
becouse it provide a service with poor quality due to the lag.

We cannot go to the cloud and instead, we do on device processing:

<img src="./images/04.png" width="700">

We have do to the data collection in order to setup the dataset on which train a model. Already we know the pipiline from the identification of requirements to the need of validation of the data collected against different problems (bias, missing variability, unbalancing, size, etc). There's another possibility that we can explore: we can **reuse existing datasets** in order to create a subset specific for our needs. We have [**ImageNet**](https://www.image-net.org/) dataset which has thousand different classes, but in our TinyML application typically we are focused one particular domain. For example, in the doorbell example, we can focus on person. So what we can do is reuse existing data with only images of persons and balance it out with other images that are not of persons. This is exactly how it was build the [**Visual Wake Words**]() dataset.

As an example, we implement a model to detect if a person is wearing a mask or not. We use a modified version of [kaggle mask dataset](https://www.kaggle.com/prasoonkottarathil/face-mask-lite-dataset). It is a dataset containing several hundreds images of person with and without mask. First we will download and extract a zip file containing the images. Then we will create a tf.data.Dataset for training and validation using the image_dataset_from_directory() utility. More about loading images in this [tutorial](https://www.tensorflow.org/tutorials/load_data/images).

In [16]:
!wget https://www.dropbox.com/s/4l5crxw0j1fwf1w/mask.zip

dyld[35241]: Library not loaded: /usr/local/opt/libunistring/lib/libunistring.2.dylib
  Referenced from: <1ECBA17E-A426-310D-9902-EFF0D9E10532> /usr/local/Cellar/wget/1.21.3/bin/wget
  Reason: tried: '/usr/local/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/usr/local/opt/libunistring/lib/libunistring.2.dylib' (no such file), '/usr/local/lib/libunistring.2.dylib' (no such file), '/usr/lib/libunistring.2.dylib' (no such file, not in dyld cache), '/usr/local/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/usr/local/Cellar/libunistring/1.1/lib/libunistring.2.dylib' (no such file), '/usr/local/lib/libunistring.2.dylib' (no such file), '/usr/lib/libunistring.2.dylib' (no such file, not in dyld cache)


In [19]:
import zipfile

with zipfile.ZipFile("data/mask.zip","r") as zip_ref:
    zip_ref.extractall("data/mask")

We have to select a model. In the context of visual data, models able to recognize things are typically large. When we look at image classification, there's lots of different neural networks architecutres.
The following graph shows the variety of different models proving information about accuracy, computing power and size: 

<img src="./images/06.png" width="700">

For example, ResNet-152 provides very high accuracy, but wew have to pay a fair bit in terms of the computational requirements, because it demands over 20 giga operations to make a single forward pass.

If we want to shrink models down, there are two fundamental things: we need to shrink down **the number of parameters** inside the model (the circle) and we need to run the model efficiently by limiting the **number of operations per second** it needs (the orizontal axis). These two together will give us a small network that can run efficiently on an embedded device.

Now we consider how to train a visual wake words model in an efficient way exploiting transfer learning